<a href="https://colab.research.google.com/github/armlabstanford/jparse/blob/main/jparse_collab_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# J-PARSE: Jacobian-based Projection Algorithm for Resolving Singularities Effectively in Inverse Kinematic Control of Serial Manipulators


### Planar Link Manipulator Task Space control
The kinematic velocity relationship between joint $q$ and end-effector $p$ is:
$${\bf \dot{p}} = J(q) {\bf \dot{q}}$$

For a two-link and three-link planar manipulator whose link lengths are:

| Link | Length |
| :-----: | :-----:  |
| $l_1$ | 1m |
| $l_2$ | 1m |
| $l_3$ | 1m |

The origin of the planar manipulator is at the origin. The end-effector must track a point that follows a circle whose center is at $(x,y)=(1,0)$, and whose radius is specified. The target position can move around the circle at a frequency of $\omega$.

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML, display
# Simulation parameters
dt = 0.05            # time step [s]
T_total = 20         # total simulation time [s]
time_steps = int(T_total/dt)
time = np.linspace(0, T_total, time_steps)

# Robot parameters
l1 = 1.0             # length of link 1 [m]
l2 = 1.0             # length of link 2 [m]

# Desired circular trajectory parameters
center = np.array([1.0, 0.0])   # center of the circle
omega = 0.25                     # angular speed for the desired trajectory [rad/s]

# Pre-allocate storage for simulation data
theta1_hist = np.zeros(time_steps)
theta2_hist = np.zeros(time_steps)
x_hist = np.zeros(time_steps)
y_hist = np.zeros(time_steps)
xd_hist = np.zeros(time_steps)
yd_hist = np.zeros(time_steps)

# INITIAL CONDITIONS: starting joint angles (could be chosen arbitrarily)
theta1 = np.pi/4
theta2 = np.pi/4


### Set Radius to visualize JParse performance

In [ ]:
singularity_bool = True
if singularity_bool == True:
  radius = 2.0 #show ability to handle singular poses
else:
  radius = 0.5 #show ability to handle non-singular poses


## J-PARSE Class
This is the J-PARSE implementation

In [ ]:
class JParseClass(object):

    def svd_compose(self,U,S,Vt):
        """
        This function takes SVD: U,S,V and recomposes them for J
        """
        Zero_concat = np.zeros((U.shape[0],Vt.shape[0]-len(S)))
        Sfull = np.zeros((U.shape[1],Vt.shape[0]))
        for row in range(Sfull.shape[0]):
            for col in range(Sfull.shape[1]):
              if row == col:
                  if row < len(S):
                      Sfull[row,col] = S[row]
        J_new =np.matrix(U)*Sfull*np.matrix(Vt)
        return J_new

    def JParse(self, J=None, jac_nullspace_bool=False, gamma=0.1, singular_direction_gain_position=1,singular_direction_gain_orientation=1, position_dimensions=None, angular_dimensions=None):
        """
        input: Jacobian J (m x n) numpy matrix
        Args:
          - jac_nullspace_bool (default=False): Set this to true of the nullspace of the jacobian is desired
          - gamma (default=0.1): threshold gain for singular directions
          - singular_direction_gain_position (default=1): gain for singular directions in position
          - singular_direction_gain_orientation (default=1): gain for singular directions in orientation
          - position_dimensions (default=None): the number of dimensions for the position *NOTE: if this is not set, then all gains are be set to the singular_directin_gain_position value
          - angular_dimensions (default=None): the number of dimensions for the orientation *NOTE: if this is not set, then all gains are be set to the singular_directin_gain_position value

        Note: if no information is provided for position or angular dimensions, then the rows of the jacobian are used to determine the gain matrix dimension, and the default (or user set) value of singular_direction_gain_position
        is used by default.

        output:
          - J_parse (n x m) numpy matrix
          - (optional) J_safety_nullspace (n x n) numpy matrix : this can be used with a secondary objective like attracting joints to a nominal position (which will not interfere with primary task objective).
                                          This is just the matrix, the joint-space objective must be calculated and multiplied by the user.
        """

        #Perform the SVD decomposition of the jacobian
        U, S, Vt = np.linalg.svd(J)
        #Find the adjusted condition number
        sigma_max = np.max(S)
        adjusted_condition_numbers = [sig / sigma_max for sig in S]

        #Find the projection Jacobian
        U_new_proj = []
        S_new_proj = []
        for col in range(len(S)):
            if S[col] > gamma*sigma_max:
                #Singular row
                U_new_proj.append(np.matrix(U[:,col]).T)
                S_new_proj.append(S[col])
        U_new_proj = np.concatenate(U_new_proj,axis=1) #Careful which numpy version is being used for this!!!!!
        J_proj = self.svd_compose(U_new_proj, S_new_proj, Vt)

        #Find the safety jacboian
        S_new_safety = [s if (s/sigma_max) > gamma else gamma*sigma_max for s in S]
        J_safety = self.svd_compose(U,S_new_safety,Vt)

        #Find the singular direction projection components
        U_new_sing = []
        Phi = [] #these will be the ratio of s_i/s_max
        set_empty_bool = True
        for col in range(len(S)):
            if adjusted_condition_numbers[col] <= gamma:
                set_empty_bool = False
                U_new_sing.append(np.matrix(U[:,col]).T)
                Phi.append(adjusted_condition_numbers[col]/gamma) #division by gamma for s/(s_max * gamma), gives smooth transition for Kp =1.0;

        #set an empty Phi_singular matrix, populate if there were any adjusted
        #condition numbers below the threshold
        Phi_singular = np.zeros(U.shape) #initialize the singular projection matrix

        if set_empty_bool == False:
            #construct the new U, as there were singular directions
            U_new_sing = np.matrix(np.concatenate(U_new_sing,axis=1)) #Careful which numpy version is being used for this!!!!!
            Phi_mat = np.matrix(np.diag(Phi))

            # Now handle the gain conditions
            if position_dimensions == None and angular_dimensions == None:
                #neither dimensions have been set, this is the default case
                gain_dimension = J.shape[0]
                gains = np.array([singular_direction_gain_position]*gain_dimension, dtype=float)
            elif angular_dimensions == None and position_dimensions != None:
                #only position dimensions have been set
                gain_dimension = position_dimensions
                gains = np.array([singular_direction_gain_position]*gain_dimension, dtype=float)
            elif position_dimensions == None and angular_dimensions != None:
                #only angular dimensions have been set
                gain_dimension = angular_dimensions
                gains = np.array([singular_direction_gain_orientation]*gain_dimension, dtype=float)
            else:
                #both position and angular dimensions are filled
                gains = np.array([singular_direction_gain_position]*position_dimensions + [singular_direction_gain_orientation]*angular_dimensions, dtype=float)
            #now put them into a matrix:
            Kp_singular = np.diag(gains)

            # Now put it all together:
            Phi_singular = U_new_sing @ Phi_mat @ U_new_sing.T @ Kp_singular

        #Obtain psuedo-inverse of the safety jacobian and the projection jacobian
        J_safety_pinv = np.linalg.pinv(J_safety)
        J_proj_pinv = np.linalg.pinv(J_proj)

        if set_empty_bool == False:
            J_parse = J_safety_pinv @ J_proj @ J_proj_pinv + J_safety_pinv @ Phi_singular
        else:
            J_parse = J_safety_pinv @ J_proj @ J_proj_pinv

        if jac_nullspace_bool == True:
            #Find the nullspace of the jacobian
            J_safety_nullspace = np.eye(J_safety.shape[1]) - J_safety_pinv @ J_safety
            return J_parse, J_safety_nullspace

        return J_parse

In [ ]:
def forward_kinematics(theta1, theta2):
    # Compute the (x, y) position of the end-effector.
    # Placeholder (using the standard 2-link planar kinematics):
    x = l1 * np.cos(theta1) + l2 * np.cos(theta1 + theta2)
    y = l1 * np.sin(theta1) + l2 * np.sin(theta1 + theta2)
    return x, y

def jacobian(theta1, theta2):
    # Compute the 2x2 Jacobian matrix for the 2-link manipulator.
    # Placeholder implementation:
    J11 = -l1 * np.sin(theta1) - l2 * np.sin(theta1 + theta2)
    J12 = -l2 * np.sin(theta1 + theta2)
    J21 =  l1 * np.cos(theta1) + l2 * np.cos(theta1 + theta2)
    J22 =  l2 * np.cos(theta1 + theta2)
    J = np.array([[J11, J12],
                  [J21, J22]])
    return J

def control_law(x_current, y_current, x_des, y_des, theta1, theta2, jparse_cls_obj=None):
    # Here is an example implementation using a proportional controller.
    k = 1.5  # proportional gain; adjust as needed
    error = np.array([x_des - x_current, y_des - y_current])
    # Desired Cartesian velocity
    v_des = k * error
    # Compute Jacobian and use pseudoinverse to get joint velocities
    J = jacobian(theta1, theta2)
    J_parse = jparse_cls_obj.JParse(J)
    dq = J_parse @ v_des
    return dq


### Simulation and Animation

In [ ]:
# Main simulation loop

J_parse_cls = JParseClass() #instantiate JPARSE class

for i, t in enumerate(time):
    # Desired end-effector position (circle)
    x_des = center[0] + radius * np.cos(omega * t)
    y_des = center[1] + radius * np.sin(omega * t)

    # Record desired trajectory
    xd_hist[i] = x_des
    yd_hist[i] = y_des

    # Get current end-effector position using forward kinematics
    x_current, y_current = forward_kinematics(theta1, theta2)
    x_hist[i] = x_current
    y_hist[i] = y_current

    # Compute joint velocities using control law
    dq = control_law(x_current, y_current, x_des, y_des, theta1, theta2, jparse_cls_obj=J_parse_cls)

    # Update joint angles
    theta1 += dq.item(0)* dt
    theta2 += dq.item(1)* dt

    # Record joint angles
    theta1_hist[i] = theta1
    theta2_hist[i] = theta2

# 1. Animate the robot arm

# Set up the figure and axis for animation
fig_anim, ax_anim = plt.subplots()
ax_anim.set_xlim(-2, 2)
ax_anim.set_ylim(-2, 2)
ax_anim.set_aspect('equal')
ax_anim.set_title("2-Link Robot Arm Tracking a Circle")

# Lines for links and points for joints
link_line, = ax_anim.plot([], [], 'o-', lw=4)
target_point, = ax_anim.plot([], [], 'rx', markersize=10)

def init():
    link_line.set_data([], [])
    target_point.set_data([], [])
    return link_line, target_point

def animate(i):
    # Compute joint positions based on current angles
    theta1_i = theta1_hist[i]
    theta2_i = theta2_hist[i]
    joint1 = np.array([l1 * np.cos(theta1_i), l1 * np.sin(theta1_i)])
    joint2 = joint1 + np.array([l2 * np.cos(theta1_i + theta2_i),
                                 l2 * np.sin(theta1_i + theta2_i)])
    # Link positions: base -> joint1 -> end-effector
    x_data = [0, joint1[0], joint2[0]]
    y_data = [0, joint1[1], joint2[1]]
    link_line.set_data(x_data, y_data)
    # Update the desired target point with sequence values
    target_point.set_data([xd_hist[i]], [yd_hist[i]])
    return link_line, target_point

ani = animation.FuncAnimation(fig_anim, animate, frames=len(time),
                              init_func=init, interval=50, blit=True)

# Display the animation as an HTML5 video (Colab compatible)
display(HTML(ani.to_jshtml()))

### Now make the necessary *plots*

In [ ]:

# 2. Plot x(t) and y(t) vs. time
fig1, ax1 = plt.subplots()
ax1.plot(time, x_hist, label='x(t) actual', linewidth=2)
ax1.plot(time, xd_hist, '--', label='x(t) desired', linewidth=2)
ax1.plot(time, y_hist, label='y(t) actual', linewidth=2)
ax1.plot(time, yd_hist, '--', label='y(t) desired', linewidth=2)
ax1.set_xlabel("Time [s]")
ax1.set_ylabel("Position [m]")
ax1.set_title("End-Effector Position vs. Time")
ax1.legend()
ax1.grid(True)
plt.show()

# 3. Plot the (x,y) trajectory in 2D
fig2, ax2 = plt.subplots()
ax2.plot(x_hist, y_hist, label='Actual Trajectory', linewidth=2)
ax2.plot(xd_hist, yd_hist, '--', label='Desired Circle', linewidth=2)
ax2.set_xlabel("x [m]")
ax2.set_ylabel("y [m]")
ax2.set_title("End-Effector Trajectory")
ax2.legend()
ax2.axis('equal')
ax2.grid(True)
plt.show()

# 4. Plot the two joint angles vs. time
fig3, ax3 = plt.subplots()
ax3.plot(time, theta1_hist, label=r'$\theta_1(t)$', linewidth=2)
ax3.plot(time, theta2_hist, label=r'$\theta_2(t)$', linewidth=2)
ax3.set_xlabel("Time [s]")
ax3.set_ylabel("Joint Angles [rad]")
ax3.set_title("Joint Angles vs. Time")
ax3.legend()
ax3.grid(True)
plt.show()

### Download plots and video

In [ ]:
# --- Save the static plots as PNG files ---
fig1.savefig('end_effector_position_vs_time.png')
fig2.savefig('end_effector_trajectory.png')
fig3.savefig('joint_angles_vs_time.png')

# --- Save the animation as an MP4 video ---
ani.save('robot_arm_animation.mp4', writer='ffmpeg')

# --- Download the files ---
from google.colab import files
files.download('end_effector_position_vs_time.png')
files.download('end_effector_trajectory.png')
files.download('joint_angles_vs_time.png')
files.download('robot_arm_animation.mp4')

# 3 Link Manipulator

In [ ]:
# --- Import Libraries ---
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML, display

# ======================
# Global Simulation Parameters
dt = 0.05            # Time step [s]
T_total = 30         # Total simulation time [s]
time_steps = int(T_total/dt)
time = np.linspace(0, T_total, time_steps)

# Robot Parameters (3-Link)
l1 = 1.0  # Link 1 length [m]
l2 = 1.0  # Link 2 length [m]
l3 = 1.0  # Link 3 length [m]

In [ ]:

def forward_kinematics(q1, q2, q3):
    """
    Compute the end-effector (x,y) position given three joint angles.

    Parameters:
        q1, q2, q3: Joint angles [rad]

    Returns:
        x, y: End-effector coordinates
    """
    x = l1 * np.cos(q1) + l2 * np.cos(q1 + q2) + l3 * np.cos(q1 + q2 + q3)
    y = l1 * np.sin(q1) + l2 * np.sin(q1 + q2) + l3 * np.sin(q1 + q2 + q3)
    return x, y

def jacobian(q1, q2, q3):
    """
    Compute the 2x3 Jacobian matrix for the 3-link planar manipulator.

    Parameters:
        q1, q2, q3: Joint angles [rad]

    Returns:
        J: 2x3 Jacobian matrix
    """
    J11 = -l1*np.sin(q1) - l2*np.sin(q1+q2) - l3*np.sin(q1+q2+q3)
    J12 = -l2*np.sin(q1+q2) - l3*np.sin(q1+q2+q3)
    J13 = -l3*np.sin(q1+q2+q3)
    J21 =  l1*np.cos(q1) + l2*np.cos(q1+q2) + l3*np.cos(q1+q2+q3)
    J22 =  l2*np.cos(q1+q2) + l3*np.cos(q1+q2+q3)
    J23 =  l3*np.cos(q1+q2+q3)
    J = np.array([[J11, J12, J13],
                  [J21, J22, J23]])
    return J

def control_law(x_current, y_current, x_des, y_des, q1, q2, q3, jacobian_type="JPARSE", jparse_cls_obj=None):
    """
    Compute the joint velocities needed to reduce the end-effector position error.
    Uses a simple proportional controller and the pseudoinverse of the Jacobian.

    Parameters:
        x_current, y_current: Current end-effector position
        x_des, y_des: Desired end-effector position
        q1, q2, q3: Current joint angles
        jacobian_type: Type of Jacobian to use for control ("Jpinv" or "J_dls")

    Returns:
        dq1, dq2, dq3: Joint velocities
    """

    k = 1.5  # Proportional gain
    error = np.array([x_des - x_current, y_des - y_current])
    v_des = k * error  # Desired Cartesian velocity
    J = jacobian(q1, q2, q3)
    if jacobian_type == "Jpinv":
      #1. Use traditinoal Jacobian pseudo-inverse
      dq = np.linalg.pinv(J) @ v_des
    elif jacobian_type == "J_dls":
      #2. Use Damped Least Squares
      lambda_dls = 0.01
      J_dls = J.T @ np.linalg.inv((J @ J.T + lambda_dls * np.eye(2)))
      dq = J_dls @ v_des
    elif jacobian_type == "JPARSE":
      J_parse = jparse_cls_obj.JParse(J)
      # equivalently one can use:
      # J_parse = jparse_cls_obj.JParse(J=J, singular_direction_gain_position=1, position_dimensions=2)

      dq = J_parse @ v_des
      ddq = [dq.item(0),dq.item(1),dq.item(2)]
      dq = ddq
    return dq[0], dq[1], dq[2]




Simulation Kinematics

In [ ]:
# ======================
# Problem 1: Trajectory Tracking
# ======================
# In Problem 1, the end-effector must track a time-varying circular trajectory.
# Two circles are considered:
#  • Part A: The desired circle is fully reachable.
#  • Part B: The desired circle has a large radius so that parts of the trajectory are challenging to track.
#
# ----------------------
# Define Trajectory Parameters
# ----------------------
# Part A: Fully reachable circle
circleA_center = np.array([1.5, 0.0])
circleA_radius = 0.5
omegaA = 0.5  # Angular speed [rad/s]

# Part B: Partially reachable circle
circleB_center = np.array([1.0, 0.0])
circleB_radius = 2.5  # Larger radius challenges the workspace limits
omegaB = 0.5

# ----------------------
# Simulation for Problem 1, Part A
# ----------------------
# Preallocate storage for simulation data for Part A
q1_hist_A = np.zeros(time_steps)
q2_hist_A = np.zeros(time_steps)
q3_hist_A = np.zeros(time_steps)
x_hist_A = np.zeros(time_steps)
y_hist_A = np.zeros(time_steps)
xd_hist_A = np.zeros(time_steps)
yd_hist_A = np.zeros(time_steps)

# Initial joint angles (can be chosen arbitrarily)
q1_A = np.pi/4
q2_A = np.pi/4
q3_A = np.pi/4

J_parse_cls = JParseClass() #instantiate JPARSE class


for i, t in enumerate(time):
    # Compute desired end-effector position for the circle trajectory:
    x_des = circleA_center[0] + circleA_radius * np.cos(omegaA * t)
    y_des = circleA_center[1] + circleA_radius * np.sin(omegaA * t)
    xd_hist_A[i] = x_des
    yd_hist_A[i] = y_des

    # Get current end-effector position using forward kinematics
    x_current, y_current = forward_kinematics(q1_A, q2_A, q3_A)
    x_hist_A[i] = x_current
    y_hist_A[i] = y_current

    # Compute joint velocities using the control law
    dq1, dq2, dq3 = control_law(x_current, y_current, x_des, y_des, q1_A, q2_A, q3_A, jparse_cls_obj=J_parse_cls)

    # Update joint angles based on the velocity-controlled system
    q1_A += dq1 * dt
    q2_A += dq2 * dt
    q3_A += dq3 * dt

    q1_hist_A[i] = q1_A
    q2_hist_A[i] = q2_A
    q3_hist_A[i] = q3_A

# ----------------------
# Simulation for Problem 1, Part B
# ----------------------
# Preallocate storage for simulation data for Part B
q1_hist_B = np.zeros(time_steps)
q2_hist_B = np.zeros(time_steps)
q3_hist_B = np.zeros(time_steps)
x_hist_B = np.zeros(time_steps)
y_hist_B = np.zeros(time_steps)
xd_hist_B = np.zeros(time_steps)
yd_hist_B = np.zeros(time_steps)

# Initial joint angles for Part B
q1_B = np.pi/4
q2_B = np.pi/4
q3_B = np.pi/4

for i, t in enumerate(time):
    # Compute desired end-effector position for the larger circle:
    x_des = circleB_center[0] + circleB_radius * np.cos(omegaB * t)
    y_des = circleB_center[1] + circleB_radius * np.sin(omegaB * t)
    xd_hist_B[i] = x_des
    yd_hist_B[i] = y_des

    # Get current end-effector position
    x_current, y_current = forward_kinematics(q1_B, q2_B, q3_B)
    x_hist_B[i] = x_current
    y_hist_B[i] = y_current

    # Compute joint velocities using the same control law
    dq1, dq2, dq3 = control_law(x_current, y_current, x_des, y_des, q1_B, q2_B, q3_B, jparse_cls_obj=J_parse_cls)

    # Update joint angles
    q1_B += dq1 * dt
    q2_B += dq2 * dt
    q3_B += dq3 * dt

    q1_hist_B[i] = q1_B
    q2_hist_B[i] = q2_B
    q3_hist_B[i] = q3_B

## 3-link animation path A

In [ ]:

# ----------------------
# Animate the 3-Link Manipulator (Problem 1, Part A)
# ----------------------
fig_anim_A, ax_anim_A = plt.subplots()
ax_anim_A.set_xlim(-3, 3)
ax_anim_A.set_ylim(-3, 3)
ax_anim_A.set_aspect('equal')
ax_anim_A.set_title("3-Link Manipulator Tracking a Reachable Circle (Part A)")

# Plot elements: manipulator links and target point
link_line_A, = ax_anim_A.plot([], [], 'o-', lw=4)
target_point_A, = ax_anim_A.plot([], [], 'rx', markersize=10)

def init_A():
    link_line_A.set_data([], [])
    target_point_A.set_data([], [])
    return link_line_A, target_point_A

def animate_A(i):
    # Compute the positions of each joint:
    joint1 = np.array([l1 * np.cos(q1_hist_A[i]), l1 * np.sin(q1_hist_A[i])])
    joint2 = joint1 + np.array([l2 * np.cos(q1_hist_A[i] + q2_hist_A[i]),
                                 l2 * np.sin(q1_hist_A[i] + q2_hist_A[i])])
    joint3 = joint2 + np.array([l3 * np.cos(q1_hist_A[i] + q2_hist_A[i] + q3_hist_A[i]),
                                 l3 * np.sin(q1_hist_A[i] + q2_hist_A[i] + q3_hist_A[i])])
    # Create a list of positions from the base (0,0) to the end-effector
    x_data = [0, joint1[0], joint2[0], joint3[0]]
    y_data = [0, joint1[1], joint2[1], joint3[1]]
    link_line_A.set_data(x_data, y_data)
    target_point_A.set_data([xd_hist_A[i]], [yd_hist_A[i]])
    return link_line_A, target_point_A

ani_A = animation.FuncAnimation(fig_anim_A, animate_A, frames=len(time),
                                init_func=init_A, interval=50, blit=True)
display(HTML(ani_A.to_jshtml()))

# Plot end-effector positions vs. time (Problem 1, Part A)
fig1_A, ax1_A = plt.subplots()
ax1_A.plot(time, x_hist_A, label='x(t) actual', linewidth=2)
ax1_A.plot(time, xd_hist_A, '--', label='x(t) desired', linewidth=2)
ax1_A.plot(time, y_hist_A, label='y(t) actual', linewidth=2)
ax1_A.plot(time, yd_hist_A, '--', label='y(t) desired', linewidth=2)
ax1_A.set_xlabel("Time [s]")
ax1_A.set_ylabel("Position [m]")
ax1_A.set_title("End-Effector Position vs. Time (Part A), using Jacobian ?? (DLS or Jpinv)")
ax1_A.legend()
ax1_A.grid(True)
plt.show()

# Plot the 2D trajectory for Part A
fig2_A, ax2_A = plt.subplots()
ax2_A.plot(x_hist_A, y_hist_A, label='Actual Trajectory', linewidth=2)
ax2_A.plot(xd_hist_A, yd_hist_A, '--', label='Desired Trajectory', linewidth=2)
ax2_A.set_xlabel("x [m]")
ax2_A.set_ylabel("y [m]")
ax2_A.set_title("End-Effector Trajectory (Part A)")
ax2_A.legend()
ax2_A.axis('equal')
ax2_A.grid(True)
plt.show()

## Animate tracking path B


In [ ]:
# ----------------------
# Animate the 3-Link Manipulator (Problem 1, Part B)
# ----------------------
fig_anim_B, ax_anim_B = plt.subplots()
ax_anim_B.set_xlim(-3, 3)
ax_anim_B.set_ylim(-3, 3)
ax_anim_B.set_aspect('equal')
ax_anim_B.set_title("3-Link Manipulator Tracking a Partially Reachable Circle (Part B)")

# Plot elements: manipulator links and target (desired) point
link_line_B, = ax_anim_B.plot([], [], 'o-', lw=4)
target_point_B, = ax_anim_B.plot([], [], 'rx', markersize=10)

def init_B():
    link_line_B.set_data([], [])
    target_point_B.set_data([], [])
    return link_line_B, target_point_B

def animate_B(i):
    # Compute positions of each joint using the Part B joint histories
    joint1 = np.array([l1 * np.cos(q1_hist_B[i]), l1 * np.sin(q1_hist_B[i])])
    joint2 = joint1 + np.array([l2 * np.cos(q1_hist_B[i] + q2_hist_B[i]),
                                 l2 * np.sin(q1_hist_B[i] + q2_hist_B[i])])
    joint3 = joint2 + np.array([l3 * np.cos(q1_hist_B[i] + q2_hist_B[i] + q3_hist_B[i]),
                                 l3 * np.sin(q1_hist_B[i] + q2_hist_B[i] + q3_hist_B[i])])
    # Create list of positions from the base (0,0) to the end-effector
    x_data = [0, joint1[0], joint2[0], joint3[0]]
    y_data = [0, joint1[1], joint2[1], joint3[1]]
    link_line_B.set_data(x_data, y_data)
    # Update the desired target point for Part B trajectory
    target_point_B.set_data([xd_hist_B[i]], [yd_hist_B[i]])
    return link_line_B, target_point_B

ani_B = animation.FuncAnimation(fig_anim_B, animate_B, frames=len(time),
                                init_func=init_B, interval=50, blit=True)
display(HTML(ani_B.to_jshtml()))

# ----------------------
# Plot end-effector positions vs. time (Part B)
# ----------------------
fig1_B, ax1_B = plt.subplots()
ax1_B.plot(time, x_hist_B, label='x(t) actual', linewidth=2)
ax1_B.plot(time, xd_hist_B, '--', label='x(t) desired', linewidth=2)
ax1_B.plot(time, y_hist_B, label='y(t) actual', linewidth=2)
ax1_B.plot(time, yd_hist_B, '--', label='y(t) desired', linewidth=2)
ax1_B.set_xlabel("Time [s]")
ax1_B.set_ylabel("Position [m]")
ax1_B.set_title("End-Effector Position vs. Time (Part B), using Jacobian Jpinv ?? (DLS or Jpinv)")
ax1_B.legend()
ax1_B.grid(True)
plt.show()

# ----------------------
# Plot the 2D trajectory for Part B
# ----------------------
fig2_B, ax2_B = plt.subplots()
ax2_B.plot(x_hist_B, y_hist_B, label='Actual Trajectory', linewidth=2)
ax2_B.plot(xd_hist_B, yd_hist_B, '--', label='Desired Trajectory', linewidth=2)
ax2_B.set_xlabel("x [m]")
ax2_B.set_ylabel("y [m]")
ax2_B.set_title("End-Effector Trajectory (Part B)")
ax2_B.legend()
ax2_B.axis('equal')
ax2_B.grid(True)
plt.show()

Download videos

In [ ]:
# ----------------------
# Save Plots and Animations
# ----------------------
# Save Problem 1, Part A plots
fig1_A.savefig('end_effector_position_vs_time_partA.png')
fig2_A.savefig('end_effector_trajectory_partA.png')

# Save Problem 1, Part B plots
fig1_B.savefig('end_effector_position_vs_time_partB.png')
fig2_B.savefig('end_effector_trajectory_partB.png')

# Save animations:
ani_A.save('manipulator_animation_partA.mp4', writer='ffmpeg')
ani_B.save('manipulator_animation_partB.mp4', writer='ffmpeg')

# --- Download files (Google Colab) ---
from google.colab import files
files.download('end_effector_position_vs_time_partA.png')
files.download('end_effector_trajectory_partA.png')
files.download('end_effector_position_vs_time_partB.png')
files.download('end_effector_trajectory_partB.png')
files.download('manipulator_animation_partA.mp4')
files.download('manipulator_animation_partB.mp4')


# 7-Link 3D Manipulator Example

As a first step, we need to download a few packages, then restart the colab computer for the pinocchio library to load - this means you will need to manually run the blocks below this first one.

In [ ]:
# 1) Download micromamba if you haven’t already
!curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xvj bin/micromamba

!pip install pin numpy matplotlib ipython -q

# 2) Create a new env with python=3.11
!./bin/micromamba create -y -p /usr/local/env \
    -c conda-forge pinocchio numpy matplotlib ipython python=3.11

# 3) Kill the process to force a clean restart
import os; os.kill(os.getpid(), 9)

A restart was needed to ensure pinocchio is accessible - so continue from the following code block

## Setup the 7-DOF 3D kinematic manipulator

In [ ]:
import numpy as np
import pinocchio as pin

# Build a 7-DOF Chain in Pinocchio
# Create model and data
model = pin.Model()
data = model.createData()

# Define 7 links of length 0.3 m, alternating joint axes Z and Y
link_lengths = [0.3] * 7
parent = 0

for i, L in enumerate(link_lengths):
    axis = i % 3  # 0 → X, 1 → Y, 2 → Z
    if axis == 0:
        jmodel = pin.JointModelRY()   # rotate about Y
    elif axis == 1:
        jmodel = pin.JointModelRX()   # rotate about X
    else:
        jmodel = pin.JointModelRZ()   # rotate about Z

    # Placement: identity rotation, translate along local x by L
    Xtree = pin.SE3(np.eye(3), np.array([L, 0, 0]))

    # Add joint
    joint_id = model.addJoint(parent, jmodel, Xtree, f"joint{i+1}")

    # Append a dummy body for the joint
    inertia = pin.Inertia(1.0, np.zeros(3), np.eye(3))
    model.appendBodyToJoint(joint_id, inertia, pin.SE3.Identity())

    parent = joint_id

# End-effector joint ID
ee_joint_id = parent

# Now create Data object sized to the model
data = model.createData()

print(f"Built model with {model.nq} DOFs and {model.njoints} joints")

## Rerun JPARSE class
We need to rerun JPARSE class again since colab was restarted when loading pinnochio. This is the same implementation as above, just copied here for convenience.

In [ ]:
class JParseClass(object):

    def svd_compose(self,U,S,Vt):
        """
        This function takes SVD: U,S,V and recomposes them for J
        """
        Zero_concat = np.zeros((U.shape[0],Vt.shape[0]-len(S)))
        Sfull = np.zeros((U.shape[1],Vt.shape[0]))
        for row in range(Sfull.shape[0]):
            for col in range(Sfull.shape[1]):
              if row == col:
                  if row < len(S):
                      Sfull[row,col] = S[row]
        J_new =np.matrix(U)*Sfull*np.matrix(Vt)
        return J_new

    def JParse(self, J=None, jac_nullspace_bool=False, gamma=0.1, singular_direction_gain_position=1,singular_direction_gain_orientation=1, position_dimensions=None, angular_dimensions=None):
        """
        input: Jacobian J (m x n) numpy matrix
        Args:
          - jac_nullspace_bool (default=False): Set this to true of the nullspace of the jacobian is desired
          - gamma (default=0.1): threshold gain for singular directions
          - singular_direction_gain_position (default=1): gain for singular directions in position
          - singular_direction_gain_orientation (default=1): gain for singular directions in orientation
          - position_dimensions (default=None): the number of dimensions for the position *NOTE: if this is not set, then all gains are be set to the singular_directin_gain_position value
          - angular_dimensions (default=None): the number of dimensions for the orientation *NOTE: if this is not set, then all gains are be set to the singular_directin_gain_position value

        Note: if no information is provided for position or angular dimensions, then the rows of the jacobian are used to determine the gain matrix dimension, and the default (or user set) value of singular_direction_gain_position
        is used by default.

        output:
          - J_parse (n x m) numpy matrix
          - (optional) J_safety_nullspace (n x n) numpy matrix : this can be used with a secondary objective like attracting joints to a nominal position (which will not interfere with primary task objective).
                                          This is just the matrix, the joint-space objective must be calculated and multiplied by the user.
        """

        #Perform the SVD decomposition of the jacobian
        U, S, Vt = np.linalg.svd(J)
        #Find the adjusted condition number
        sigma_max = np.max(S)
        adjusted_condition_numbers = [sig / sigma_max for sig in S]

        #Find the projection Jacobian
        U_new_proj = []
        S_new_proj = []
        for col in range(len(S)):
            if S[col] > gamma*sigma_max:
                #Singular row
                U_new_proj.append(np.matrix(U[:,col]).T)
                S_new_proj.append(S[col])
        U_new_proj = np.concatenate(U_new_proj,axis=1) #Careful which numpy version is being used for this!!!!!
        J_proj = self.svd_compose(U_new_proj, S_new_proj, Vt)

        #Find the safety jacboian
        S_new_safety = [s if (s/sigma_max) > gamma else gamma*sigma_max for s in S]
        J_safety = self.svd_compose(U,S_new_safety,Vt)

        #Find the singular direction projection components
        U_new_sing = []
        Phi = [] #these will be the ratio of s_i/s_max
        set_empty_bool = True
        for col in range(len(S)):
            if adjusted_condition_numbers[col] <= gamma:
                set_empty_bool = False
                U_new_sing.append(np.matrix(U[:,col]).T)
                Phi.append(adjusted_condition_numbers[col]/gamma) #division by gamma for s/(s_max * gamma), gives smooth transition for Kp =1.0;

        #set an empty Phi_singular matrix, populate if there were any adjusted
        #condition numbers below the threshold
        Phi_singular = np.zeros(U.shape) #initialize the singular projection matrix

        if set_empty_bool == False:
            #construct the new U, as there were singular directions
            U_new_sing = np.matrix(np.concatenate(U_new_sing,axis=1)) #Careful which numpy version is being used for this!!!!!
            Phi_mat = np.matrix(np.diag(Phi))

            # Now handle the gain conditions
            if position_dimensions == None and angular_dimensions == None:
                #neither dimensions have been set, this is the default case
                gain_dimension = J.shape[0]
                gains = np.array([singular_direction_gain_position]*gain_dimension, dtype=float)
            elif angular_dimensions == None and position_dimensions != None:
                #only position dimensions have been set
                gain_dimension = position_dimensions
                gains = np.array([singular_direction_gain_position]*gain_dimension, dtype=float)
            elif position_dimensions == None and angular_dimensions != None:
                #only angular dimensions have been set
                gain_dimension = angular_dimensions
                gains = np.array([singular_direction_gain_orientation]*gain_dimension, dtype=float)
            else:
                #both position and angular dimensions are filled
                gains = np.array([singular_direction_gain_position]*position_dimensions + [singular_direction_gain_orientation]*angular_dimensions, dtype=float)
            #now put them into a matrix:
            Kp_singular = np.diag(gains)

            # Now put it all together:
            Phi_singular = U_new_sing @ Phi_mat @ U_new_sing.T @ Kp_singular

        #Obtain psuedo-inverse of the safety jacobian and the projection jacobian
        J_safety_pinv = np.linalg.pinv(J_safety)
        J_proj_pinv = np.linalg.pinv(J_proj)

        if set_empty_bool == False:
            J_parse = J_safety_pinv @ J_proj @ J_proj_pinv + J_safety_pinv @ Phi_singular
        else:
            J_parse = J_safety_pinv @ J_proj @ J_proj_pinv

        if jac_nullspace_bool == True:
            #Find the nullspace of the jacobian
            J_safety_nullspace = np.eye(J_safety.shape[1]) - J_safety_pinv @ J_safety
            return J_parse, J_safety_nullspace

        return J_parse

## Trajectory & Control Loop (Position Control ONLY)
Now that we have a 7dof, 3D manipulator, lets establish a helical trajectory, and track just the position of the end-effector

In [ ]:
import matplotlib.pyplot as plt

# Time parameters
dt = 0.05
T_total = 30.0
time = np.arange(0, T_total, dt)

# Desired 3D helix
radius  = 0.5
z_speed = 0.05
omega   = 0.1

pos_des = np.array([
    [radius*np.cos(omega*t),
     radius*np.sin(omega*t),
     z_speed*t]
    for t in time
])

# Initialize
q      = np.zeros(model.nq)
q_hist = np.zeros((len(time), model.nq))
x_hist = np.zeros((len(time), 3))
k      = 2.0  # control gain

J_parse_cls = JParseClass() #instantiate JPARSE class

for i, t in enumerate(time):
    # 1) Forward kinematics
    pin.forwardKinematics(model, data, q)
    pin.updateFramePlacements(model, data)

    # 2) End-effector position from the last joint
    ee_pos = data.oMi[ee_joint_id].translation

    # 3) Jacobian of that joint (6×nq), then take the top 3 rows
    J6 = pin.computeJointJacobian(model, data, q, ee_joint_id)
    J  = J6[:3, :] #pick just the position components of the jacobian

    # JPARSE
    J_parse = J_parse_cls.JParse(J)

    # 4) Velocity control
    error = pos_des[i] - ee_pos
    # dq    = np.linalg.pinv(J) @ (k * error) #uncomment this and comment below if you want to see a traditional jacobian pseudo-inverse
    dq    = J_parse @ (k * error)
    dq = dq.A1 #convert from matrix to ndarray

    # 5) Integrate & store
    q       += dq * dt
    q_wrapped = (q + np.pi) % (2*np.pi) - np.pi

    q_hist[i] = q_wrapped
    x_hist[i] = ee_pos

### Animate (position only) in Plotly for interactivity

* 3D Animation tracking only position in 3D for a 7DOF arm


* To interact with the plot, pause running then adjust orientation/position of the view then resume with the play button

In [ ]:
# Install Plotly if needed
!pip install plotly -q

import numpy as np
import plotly.graph_objects as go

# Helper to get positions & orientation
def fk_pin(q):
    pin.forwardKinematics(model, data, q)
    pin.updateFramePlacements(model, data)
    # collect base + each joint placement up to EE
    pts = [data.oMi[j].translation for j in range(ee_joint_id+1)]
    R   = data.oMi[ee_joint_id].rotation
    return pts, R

# Build the initial traces (frame 0)
pts0, R0      = fk_pin(q_hist[0])
xs0, ys0, zs0 = zip(*pts0)
ee0           = np.array(pts0[-1])
triad_len     = 0.1
x0 = ee0 + R0[:,0]*triad_len
y0 = ee0 + R0[:,1]*triad_len
z0 = ee0 + R0[:,2]*triad_len

target_marker_size = 8

link0 = go.Scatter3d(
    x=xs0, y=ys0, z=zs0,
    mode='lines+markers',
    marker=dict(size=4),
    line=dict(width=6, color='gray'),
    name='Links'
)
target0 = go.Scatter3d(
    x=[pos_des[0,0]], y=[pos_des[0,1]], z=[pos_des[0,2]],
    mode='markers',
    marker=dict(size=target_marker_size, color='magenta', symbol='circle'),
    name='Desired EE'
)
xtri0 = go.Scatter3d(
    x=[ee0[0],x0[0]], y=[ee0[1],x0[1]], z=[ee0[2],x0[2]],
    mode='lines', line=dict(color='red',   width=4), name='X-axis'
)
ytri0 = go.Scatter3d(
    x=[ee0[0],y0[0]], y=[ee0[1],y0[1]], z=[ee0[2],y0[2]],
    mode='lines', line=dict(color='green', width=4), name='Y-axis'
)
ztri0 = go.Scatter3d(
    x=[ee0[0],z0[0]], y=[ee0[1],z0[1]], z=[ee0[2],z0[2]],
    mode='lines', line=dict(color='blue',  width=4), name='Z-axis'
)

# Build animation frames
frames = []
for i in range(len(time)):
    pts, R = fk_pin(q_hist[i])
    xs, ys, zs = zip(*pts)
    ee = np.array(pts[-1])
    x_ax = ee + R[:,0]*triad_len
    y_ax = ee + R[:,1]*triad_len
    z_ax = ee + R[:,2]*triad_len

    frames.append(go.Frame(
        data=[
            go.Scatter3d(x=xs, y=ys, z=zs),
            go.Scatter3d(x=[pos_des[i,0]], y=[pos_des[i,1]], z=[pos_des[i,2]]),
            go.Scatter3d(x=[ee[0],x_ax[0]], y=[ee[1],x_ax[1]], z=[ee[2],x_ax[2]]),
            go.Scatter3d(x=[ee[0],y_ax[0]], y=[ee[1],y_ax[1]], z=[ee[2],y_ax[2]]),
            go.Scatter3d(x=[ee[0],z_ax[0]], y=[ee[1],z_ax[1]], z=[ee[2],z_ax[2]])
        ],
        name=str(i)
    ))

# Create the figure with play button
fig = go.Figure(
    data=[link0, target0, xtri0, ytri0, ztri0],
    layout=go.Layout(
        title="7-DOF Arm with Target Pose (Pinocchio FK)",
        scene=dict(
            xaxis=dict(range=[-2.5, 2.5], autorange=False, title="X"),
            yaxis=dict(range=[-2.5, 2.5], autorange=False, title="Y"),
            zaxis=dict(range=[-2.5, 2.5], autorange=False, title="Z"),
            aspectmode='cube'
        ),
        updatemenus=[dict(
            type="buttons",
            showactive=False,
            x=0.1, y=0,
            buttons=[
                dict(
                    label="Play",
                    method="animate",
                    args=[None, {
                        "frame": {"duration": 40, "redraw": True},
                        "fromcurrent": True,
                        "transition": {"duration": 0}
                    }]
                ),
                dict(
                    label="Pause",
                    method="animate",
                    args=[
                        [None],
                        {
                            "frame": {"duration": 0, "redraw": False},
                            "mode": "immediate",
                            "transition": {"duration": 0}
                        }
                    ]
                )
            ]
        )]
    ),
    frames=frames
)
# Display the interactive animation
fig.show()

# For tracking full 3D pose (position and orientation)
The previous example only tracked position, this example tracks both position and orientation (a static orientation which must be controlled).

In [ ]:
# Full SE(3) Pose–Level Control Using JParse

import numpy as np
import pinocchio as pin

def rotation_matrix_to_axis_angle(R):
    """Convert rotation matrix to axis-angle vector."""
    cos_theta = (np.trace(R) - 1) / 2.0
    theta = np.arccos(np.clip(cos_theta, -1.0, 1.0))
    if np.isclose(theta, 0.0):
        return np.zeros(3)
    if np.isclose(theta, np.pi):
        axis = np.sqrt((np.diag(R) + 1) / 2.0)
        axis *= np.sign([R[2,1] - R[1,2],
                         R[0,2] - R[2,0],
                         R[1,0] - R[0,1]])
        return axis * theta
    axis = np.array([
        R[2,1] - R[1,2],
        R[0,2] - R[2,0],
        R[1,0] - R[0,1]
    ]) / (2.0 * np.sin(theta))
    return axis * theta

# Time parameters
dt      = 0.05
T_total = 30.0
time    = np.arange(0.0, T_total, dt)

# Desired 3D helix (position only; keep orientation fixed)
radius  = 0.5
z_speed = 0.05
omega   = 0.1
pos_des = np.array([
    [ radius * np.cos(omega * t),
      radius * np.sin(omega * t),
      z_speed * t ]
    for t in time
])

# Storage
q      = np.zeros(model.nq)            # initial joint config
q_hist = np.zeros((len(time), model.nq))
x_hist = np.zeros((len(time), 3))

# Gains
k_pos = 1.5   # position gain
k_rot = 1.5   # orientation gain

# Jparse
J_parse_cls = JParseClass()


for i, t in enumerate(time):
    # 1) Forward kinematics
    pin.forwardKinematics(model, data, q)
    pin.updateFramePlacements(model, data)
    # 2) Compute joint Jacobians
    pin.computeJointJacobians(model, data, q)

    # 3) Read current EE placement from data.oMi (joint placement)
    oMi   = data.oMi[ee_joint_id]
    p_cur = oMi.translation            # current position (3,)
    R_cur = oMi.rotation               # current orientation (3×3)

    # 4) Desired EE pose (here position from helix, identity orientation)
    p_des = pos_des[i]
    R_des = np.eye(3)

    # 5) Compute pose errors
    e_pos = p_des - p_cur              # translational error (3,)
    R_err = R_des @ R_cur.T            # orientation error matrix
    e_rot = rotation_matrix_to_axis_angle(R_err)  # axis-angle (3,)

    # 6) Build 6×nv Jacobian: first 3 rows = translational, next 3 = rotational
    J6 = data.J                        # shape = (6, nv)

    # J_parse = J_parse_cls.JParse(J6)
    J_parse = J_parse_cls.JParse(J=J6, singular_direction_gain_position=1,singular_direction_gain_orientation=2, position_dimensions=3, angular_dimensions=3)


    # 7) Desired spatial velocity
    v_spatial = np.hstack((k_pos * e_pos, k_rot * e_rot))  # (6,)

    # 8) Solve for joint velocities (nv,)
    # dq = np.linalg.pinv(J6) @ v_spatial #this is a regular jacobian psuedo-inverse
    dq = J_parse @ v_spatial
    dq = dq.A1 #convert from matrix to ndarray

    # 9) Integrate (fixed-base)
    q += dq * dt
    q_wrapped = (q + np.pi) % (2*np.pi) - np.pi

    # 10) Record history
    q_hist[i] = q_wrapped
    # q_hist[i] = q
    x_hist[i] = p_cur


### Plot the error versus time

In [ ]:
# === Block 3: Plot Position, Orientation Error, AND Joint Angles vs. Time ===

import matplotlib.pyplot as plt

# Recompute or load your error histories
pos_err = pos_des - x_hist                                 # (N×3)

rot_err = np.zeros_like(pos_err)                           # (N×3)
for i in range(len(time)):
    pin.forwardKinematics(model, data, q_hist[i])
    pin.updateFramePlacements(model, data)
    pin.computeJointJacobians(model, data, q_hist[i])
    oMi   = data.oMi[ee_joint_id]
    R_cur = oMi.rotation
    R_err = np.eye(3) @ R_cur.T
    rot_err[i] = rotation_matrix_to_axis_angle(R_err)

# Create a 3‐row subplot
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 12), sharex=True)

# 1) Position errors
ax1.plot(time, pos_err[:, 0], label='Error X', linewidth=2)
ax1.plot(time, pos_err[:, 1], label='Error Y', linewidth=2)
ax1.plot(time, pos_err[:, 2], label='Error Z', linewidth=2)
ax1.set_ylabel("Position Error [m]")
ax1.set_title("End-Effector Position Error vs. Time")
ax1.legend(loc='upper right')
ax1.grid(True)

# 2) Orientation errors
ax2.plot(time, rot_err[:, 0], label='Error Roll (X)', linewidth=2)
ax2.plot(time, rot_err[:, 1], label='Error Pitch (Y)', linewidth=2)
ax2.plot(time, rot_err[:, 2], label='Error Yaw (Z)', linewidth=2)
ax2.set_ylabel("Orientation Error [rad]")
ax2.set_title("End-Effector Orientation Error vs. Time")
ax2.legend(loc='upper right')
ax2.grid(True)

# 3) Joint angles
for j in range(q_hist.shape[1]):
    ax3.plot(time, q_hist[:, j], label=f'Joint {j+1}', linewidth=1.5)
ax3.set_xlabel("Time [s]")
ax3.set_ylabel("Joint Angle [rad]")
ax3.set_title("Joint Angles vs. Time")
ax3.legend(loc='upper right', ncol=2)
ax3.grid(True)

plt.tight_layout()
plt.show()

## Use plotly to animate 3D full pose tracking
To interact with the plot, pause running then adjust orientation/position of the view then resume with the play button

In [ ]:
# === Updated Plotly Animation Using Pinocchio FK (Script 1) ===

!pip install plotly -q

import numpy as np
import plotly.graph_objects as go
import pinocchio as pin

# assume: model, data, ee_joint_id, q_hist, pos_des, time are in scope
triad_len = 0.1
target_marker_size = 8

def fk_pin(q):
    pin.forwardKinematics(model, data, q)
    pin.updateFramePlacements(model, data)
    # collect base + each joint placement up to EE
    pts = [data.oMi[j].translation for j in range(ee_joint_id+1)]
    R   = data.oMi[ee_joint_id].rotation
    return pts, R

# initial frame (i = 0)
pts0, R0    = fk_pin(q_hist[0])
xs0, ys0, zs0 = zip(*pts0)
ee0         = np.array(pts0[-1])

# target pose triad endpoints
td0         = pos_des[0]
tx0 = td0 + np.eye(3)[:,0]*triad_len
ty0 = td0 + np.eye(3)[:,1]*triad_len
tz0 = td0 + np.eye(3)[:,2]*triad_len

link0 = go.Scatter3d(x=xs0, y=ys0, z=zs0,
                     mode='lines+markers',
                     marker=dict(size=4, color='black'),
                     line=dict(width=6, color='gray'),
                     name='Links')
target0 = go.Scatter3d(x=[td0[0]], y=[td0[1]], z=[td0[2]],
                       mode='markers',
                       marker=dict(size=target_marker_size, color='magenta', symbol='circle'),
                       name='Target')
# EE triad
xtri0 = go.Scatter3d(x=[ee0[0], ee0[0]+R0[:,0][0]*triad_len],
                     y=[ee0[1], ee0[1]+R0[:,0][1]*triad_len],
                     z=[ee0[2], ee0[2]+R0[:,0][2]*triad_len],
                     mode='lines', line=dict(color='red',   width=6), name='EE X')
ytri0 = go.Scatter3d(x=[ee0[0], ee0[0]+R0[:,1][0]*triad_len],
                     y=[ee0[1], ee0[1]+R0[:,1][1]*triad_len],
                     z=[ee0[2], ee0[2]+R0[:,1][2]*triad_len],
                     mode='lines', line=dict(color='green', width=6), name='EE Y')
ztri0 = go.Scatter3d(x=[ee0[0], ee0[0]+R0[:,2][0]*triad_len],
                     y=[ee0[1], ee0[1]+R0[:,2][1]*triad_len],
                     z=[ee0[2], ee0[2]+R0[:,2][2]*triad_len],
                     mode='lines', line=dict(color='blue',  width=6), name='EE Z')
# Target triad
txtri0 = go.Scatter3d(x=[td0[0], tx0[0]],
                      y=[td0[1], tx0[1]],
                      z=[td0[2], tx0[2]],
                      mode='lines', line=dict(color='red',   width=4), name='Tgt X')
tytri0 = go.Scatter3d(x=[td0[0], ty0[0]],
                      y=[td0[1], ty0[1]],
                      z=[td0[2], ty0[2]],
                      mode='lines', line=dict(color='green', width=4), name='Tgt Y')
tztri0 = go.Scatter3d(x=[td0[0], tz0[0]],
                      y=[td0[1], tz0[1]],
                      z=[td0[2], tz0[2]],
                      mode='lines', line=dict(color='blue',  width=4), name='Tgt Z')

frames = []
for i in range(len(time)):
    pts, R   = fk_pin(q_hist[i])
    xs, ys, zs = zip(*pts)
    ee       = np.array(pts[-1])
    td       = pos_des[i]
    tx = td + np.eye(3)[:,0]*triad_len
    ty = td + np.eye(3)[:,1]*triad_len
    tz = td + np.eye(3)[:,2]*triad_len

    frames.append(go.Frame(
        data=[
            go.Scatter3d(x=xs, y=ys, z=zs),
            go.Scatter3d(x=[td[0]], y=[td[1]], z=[td[2]],
                         mode='markers',
                         marker=dict(size=target_marker_size, color='magenta', symbol='circle')),
            go.Scatter3d(x=[ee[0], ee[0]+R[:,0][0]*triad_len],
                         y=[ee[1], ee[1]+R[:,0][1]*triad_len],
                         z=[ee[2], ee[2]+R[:,0][2]*triad_len],
                         mode='lines', line=dict(color='red',   width=6)),
            go.Scatter3d(x=[ee[0], ee[0]+R[:,1][0]*triad_len],
                         y=[ee[1], ee[1]+R[:,1][1]*triad_len],
                         z=[ee[2], ee[2]+R[:,1][2]*triad_len],
                         mode='lines', line=dict(color='green', width=6)),
            go.Scatter3d(x=[ee[0], ee[0]+R[:,2][0]*triad_len],
                         y=[ee[1], ee[1]+R[:,2][1]*triad_len],
                         z=[ee[2], ee[2]+R[:,2][2]*triad_len],
                         mode='lines', line=dict(color='blue',  width=6)),
            go.Scatter3d(x=[td[0], tx[0]], y=[td[1], tx[1]], z=[td[2], tx[2]],
                         mode='lines', line=dict(color='red',   width=4)),
            go.Scatter3d(x=[td[0], ty[0]], y=[td[1], ty[1]], z=[td[2], ty[2]],
                         mode='lines', line=dict(color='green', width=4)),
            go.Scatter3d(x=[td[0], tz[0]], y=[td[1], tz[1]], z=[td[2], tz[2]],
                         mode='lines', line=dict(color='blue',  width=4))
        ],
        name=str(i)
    ))

# Add both Play and Pause buttons to the updatemenu
fig = go.Figure(
    data=[link0, target0, xtri0, ytri0, ztri0, txtri0, tytri0, tztri0],
    layout=go.Layout(
        title="7-DOF Arm with Target Pose (Pinocchio FK)",
        scene=dict(
            xaxis=dict(range=[-2.5, 2.5], autorange=False, title="X"),
            yaxis=dict(range=[-2.5, 2.5], autorange=False, title="Y"),
            zaxis=dict(range=[-2.5, 2.5], autorange=False, title="Z"),
            aspectmode='cube'
        ),
        updatemenus=[dict(
            type="buttons",
            showactive=False,
            x=0.1, y=0,
            buttons=[
                dict(
                    label="Play",
                    method="animate",
                    args=[None, {
                        "frame": {"duration": 40, "redraw": True},
                        "fromcurrent": True,
                        "transition": {"duration": 0}
                    }]
                ),
                dict(
                    label="Pause",
                    method="animate",
                    args=[
                        [None],
                        {
                            "frame": {"duration": 0, "redraw": False},
                            "mode": "immediate",
                            "transition": {"duration": 0}
                        }
                    ]
                )
            ]
        )]
    ),
    frames=frames
)

fig.show()